# CLEAN BUILD FROM SCRATCH EFFNET-UNET b7 on patches

In [1]:
CURRENT_FOLD = 1

# imports

In [2]:
import random

import numpy as np
import pandas as pd
import cv2 as cv
import os

from matplotlib import pyplot as plt
from glob import glob
from sklearn.model_selection import StratifiedKFold
from tqdm import tqdm
from IPython.display import clear_output

import albumentations as A
import albumentations.pytorch

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset
from torch.utils.data import DataLoader

In [3]:
from customeunet.utils import *
from customeunet.layers import *
from customeunet.efficientnet import *
from customeunet.efficientunet import *

# model

In [4]:
def AUXhead(in_channels, mid_channels, out_channels, upsample=2):
    return nn.Sequential(
        nn.Conv2d(in_channels, mid_channels, kernel_size=3, stride=1, padding=1),
        nn.BatchNorm2d(mid_channels),
        nn.ReLU(inplace=True),
        nn.Conv2d(mid_channels, out_channels, kernel_size=3, stride=1, padding=1),
        nn.Upsample(scale_factor=upsample, mode='bilinear')
    )

In [5]:
class EfficientUnetAUX(nn.Module):
    def __init__(self, backbone, n_classes=2, concat_input=True):
        super().__init__()
        
        self.backbone = backbone
        
        self.encoder = EfficientNet.encoder(backbone, pretrained=True)
        self.model = EfficientUnet(self.encoder, out_channels=n_classes, concat_input=True)
        
        self.aux3head = AUXhead(128, 256, n_classes, 4)
        self.aux2head = AUXhead(256, 256, n_classes, 8)
        
    def forward(self, x):
        data = self.model(x)
        aux2 = self.aux2head(data['aux2'])
        aux3 = self.aux3head(data['aux3'])
        out = {
            'logits': data['x'],
            'aux2': aux2,
            'aux3': aux3,
        }
        return out

In [6]:
model = EfficientUnetAUX(backbone='efficientnet-b7', n_classes=6)

# data

In [7]:
ORGAN2ID = {
    'kidney' : 0,
    'prostate' : 1,
    'largeintestine' : 2,
    'spleen' : 3,
    'lung' : 4
}
ID2ORGAN = {v: k for k, v in ORGAN2ID.items()}

## data 512

In [8]:
# img256 = cv.imread('./../data/train_data_eunet_patches/256/train/10044_0000.png')
# img512 = cv.imread('./../data/train_data_eunet_patches/512/train/10044_0000.png')

In [9]:
TRAIN_CSV = './../data/train.csv'
TEST_CSV = './../data/test.csv'
TRAIN_IMAGES_DIR = './../data/train_data_eunet_patches/512/train/'
TRAIN_MASKS_DIR = './../data/train_data_eunet_patches/512/masks/'

N_CROSS_VALID_SPLITS = 5
RANDOM_SEED = 2022
CURRENT_FOLD = 1

SPATIAL_SIZE = 512 # размер патча


In [10]:
train_df = pd.read_csv(TRAIN_CSV)

In [11]:
def make_train_image_path(row):
    return os.path.join(TRAIN_IMAGES_DIR, str(row['id']) + '.png')

def make_train_mask_path(row):
    return os.path.join(TRAIN_MASKS_DIR, str(row['id']) + '.png')

def create_folds(df: pd.DataFrame, n_splits: int, random_seed: int) -> pd.DataFrame:
    skf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=random_seed)
    for fold, (_, val_idx) in enumerate(skf.split(X=df, y=df["organ"])):
        df.loc[val_idx, "fold"] = int(fold)

    return df

In [12]:
train_df = create_folds(train_df, n_splits=N_CROSS_VALID_SPLITS, random_seed=RANDOM_SEED)

In [13]:
def make_512_dataset(df):
    new_df = []
    for index, row in df.iterrows():
        id = row['id']
        glb = list(glob(TRAIN_IMAGES_DIR + '{}_*.png'.format(id)))
        for fname in glb:
            newrow = dict()
            newrow['id'] = id
            newrow['image'] = fname
            newrow['mask'] = fname.replace('/train/', '/masks/')
            newrow['organ'] = row['organ']
            newrow['pixel_size'] = row['pixel_size']
            newrow['fold'] = row['fold']
            
            new_df.append(newrow)
    return pd.DataFrame(new_df)

In [14]:
train_df_512 = make_512_dataset(train_df)
print(train_df_512.shape)
train_df_512.head()

(2895, 6)


,fold,id,image,mask,organ,pixel_size
0,4.0,10044,./../data/train_data_eunet_patches/512/train/1...,./../data/train_data_eunet_patches/512/masks/1...,prostate,0.4
1,4.0,10044,./../data/train_data_eunet_patches/512/train/1...,./../data/train_data_eunet_patches/512/masks/1...,prostate,0.4
2,4.0,10044,./../data/train_data_eunet_patches/512/train/1...,./../data/train_data_eunet_patches/512/masks/1...,prostate,0.4
3,4.0,10044,./../data/train_data_eunet_patches/512/train/1...,./../data/train_data_eunet_patches/512/masks/1...,prostate,0.4
4,4.0,10044,./../data/train_data_eunet_patches/512/train/1...,./../data/train_data_eunet_patches/512/masks/1...,prostate,0.4


In [15]:
mean = np.array([0.7720342, 0.74582646, 0.76392896])
std = np.array([0.24745085, 0.26182273, 0.25782376])

In [16]:
# train_transform= A.Compose([
#     A.Resize(512, 512, interpolation=cv.INTER_LINEAR),
#     A.HorizontalFlip(p=0.5),
#     A.VerticalFlip(p=0.5),
#     A.HueSaturationValue(p=0.2),
#     A.ShiftScaleRotate(shift_limit=0.02, scale_limit=(0.05, 0.6),rotate_limit=90, p=0.5),
#     A.ElasticTransform(p=0.1, alpha=90, sigma=120 * 0.05, alpha_affine=120 * 0.03),
#     A.GridDistortion(p=0.25),
#     A.Blur(blur_limit=5, p=0.1),
#     A.GaussNoise(var_limit=(20, 100), p=0.2),
#     A.ChannelDropout(p=0.05),
#     A.RandomGamma(p=0.1),
#     A.RandomBrightnessContrast(p=0.2),
#     A.Normalize(mean=mean, std=std, 
#                 max_pixel_value=255, always_apply=True),
#     albumentations.pytorch.ToTensorV2()
# ], p=1)

In [17]:
test_transform = A.Compose([
    A.Normalize(mean=mean, std=std, 
                 max_pixel_value=255, always_apply=True),
    albumentations.pytorch.ToTensorV2()
], p=1)

In [19]:
def worker_init_fn(worker_id):
    torch_seed = torch.initial_seed()
    random.seed(torch_seed + worker_id)
    if torch_seed >= 2**30:  # make sure torch_seed + workder_id < 2**32
        torch_seed = torch_seed % 2**30
    np.random.seed(torch_seed + worker_id)
    random.seed(torch_seed + worker_id + 77777)

# VALID

In [41]:
def dice_coef2(y_true, y_pred):
    y_true_f = y_true.flatten()
    y_pred_f = y_pred.flatten()
    union = np.sum(y_true_f) + np.sum(y_pred_f)
    if union < 1: 
        return 1
    intersection = np.sum(y_true_f * y_pred_f)
#     print('UNION', union)
    return 2. * intersection / union

In [21]:
device = 'cuda:1'
DEVICE = device

In [22]:
model = EfficientUnetAUX(backbone='efficientnet-b7', n_classes=6).to(device)

In [76]:
model.load_state_dict(torch.load('./checkpoint_fold_1/effunet_ep_085_dice_00000nan_loss_0.062776.pt', 
                                 map_location=device))

<All keys matched successfully>

In [77]:
_ = model.eval()

In [78]:
import monai
from monai.utils.enums import BlendMode, PytorchPadMode
from monai.inferers import sliding_window_inference

In [79]:
class Predictor(nn.Module):
    def __init__(self, model):
        super().__init__()
        self.model = model
    
    def forward(self, x):
        out = self.model(x)
        prob = F.softmax(out['logits'], dim=1)
        return prob

In [80]:
predictor = Predictor(model)

In [81]:
train_df[train_df['fold'] == 1].head()

,id,organ,data_source,img_height,img_width,pixel_size,tissue_thickness,rle,age,sex,fold
10,10912,prostate,HPA,3000,3000,0.4,4,1147323 50 1150316 59 1153309 68 1156303 76 11...,60.0,Male,1.0
15,11448,spleen,HPA,3000,3000,0.4,4,3450282 28 3453278 33 3456272 53 3459271 56 34...,21.0,Female,1.0
16,11497,kidney,HPA,3000,3000,0.4,4,940436 51 943430 65 946427 76 949423 89 952417...,41.0,Female,1.0
22,1184,prostate,HPA,3000,3000,0.4,4,1916384 12 1919383 15 1922378 26 1925376 29 19...,60.0,Male,1.0
26,1220,lung,HPA,3000,3000,0.4,4,1429797 16 1432793 21 1435791 23 1438789 27 14...,59.0,Male,1.0


In [82]:
def infer_id(predictor, id: str, organ: str, pixel_size=0.4):
    THRESHOLD = 0.5
    
    image = cv.imread('./../data/train_images/{}.tiff'.format(id))
    image = cv.cvtColor(image, cv.COLOR_BGR2RGB)
    
    gt_mask  = np.load('./../data/train_masks/{}.npy'.format(id))
    
    scale_factor = 0.5 * (pixel_size / 0.4)
    image = cv.resize(image, None, fx=scale_factor, fy=scale_factor, interpolation=cv.INTER_AREA)
    
    image_rot = image.copy()
    masks = []
    for _ in range(4):
        image_rot = cv.rotate(image, cv.ROTATE_90_CLOCKWISE)
        
        image_tensor = test_transform(image=image)['image']

        image_tensor = image_tensor.unsqueeze(0).to(device)

        with torch.no_grad():
            infer_result = sliding_window_inference(
                image_tensor, roi_size=(512, 512), sw_batch_size=2, predictor=predictor, 
                overlap=0.25, mode=BlendMode.CONSTANT, sigma_scale=0.125, 
                padding_mode=PytorchPadMode.CONSTANT, cval=0.0,       
            )

        organ_id = ORGAN2ID[organ] + 1
        pr_mask = infer_result.detach().cpu().numpy()[0, organ_id]
        masks.append(pr_mask)
        
    pr_mask = (np.array(masks).mean(axis=0) > THRESHOLD).astype(np.uint8)
#     print(gt_mask.shape, gt_mask.dtype, pr_mask.dtype)
    pr_mask = cv.resize(pr_mask, gt_mask.shape, cv.INTER_NEAREST)
    
    return pr_mask, gt_mask

In [83]:
val_df = train_df[train_df['fold'] == 1]

In [84]:
dices = []

by_organ_dice = {key:[] for key in ORGAN2ID.keys()}
for index, row in tqdm(val_df.iterrows(), total=len(val_df)):
    id = row['id']
    organ = row['organ']
    pixel_size = row['pixel_size']
    pred_mask, gt_mask = infer_id(predictor, id, organ=organ, pixel_size=pixel_size)
    dice = dice_coef2(gt_mask, pred_mask)
    dices.append(dice)
    by_organ_dice[organ].append(dice)

  0%|          | 0/70 [00:00<?, ?it/s]/opt/conda/lib/python3.6/site-packages/torch/nn/functional.py:3635: UserWarning: Default upsampling behavior when mode=bilinear is changed to align_corners=False since 0.4.0. Please specify align_corners=True if the old behavior is desired. See the documentation of nn.Upsample for details.
  "See the documentation of nn.Upsample for details.".format(mode)
100%|██████████| 70/70 [02:52<00:00,  2.46s/it]


In [85]:
print(np.mean(dices))
for key, value in by_organ_dice.items():
    print('{:20}: {:6.5f}'.format(key, np.mean(value)))

0.7760055727847982
kidney              : 0.95051
prostate            : 0.82773
largeintestine      : 0.89371
spleen              : 0.81414
lung                : 0.16111


In [75]:
print(np.mean(dices))
for key, value in by_organ_dice.items():
    print('{:20}: {:6.5f}'.format(key, np.mean(value)))

0.7745120943796698
kidney              : 0.95295
prostate            : 0.81402
largeintestine      : 0.89369
spleen              : 0.80159
lung                : 0.18441


#### one example

In [58]:
pred_mask, gt_mask = infer_id(predictor, '10912', organ='prostate', pixel_size=0.4)

In [60]:
dice_coef2(gt_mask, pred_mask)

0.6158939680755389